# 1.) Import the Credit Card Fraud Data From CCLE

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from google.colab import drive

In [5]:
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [7]:
drive.mount('/content/gdrive/', force_remount = True)

Mounted at /content/gdrive/


In [8]:
df_tot = pd.read_csv("gdrive/MyDrive/ML/fraudTest.csv")

In [9]:
df_tot.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


# 2.) Select four columns to use as features (one just be trans_date_trans)

In [10]:
df = df_tot[["trans_date_trans_time", "gender", "amt", "city_pop", "is_fraud"]]

In [11]:
df.columns

Index(['trans_date_trans_time', 'gender', 'amt', 'city_pop', 'is_fraud'], dtype='object')

# 3.) Create a your own variable out of trans_date. Create dummies for factor vars

In [12]:
type(df["trans_date_trans_time"][0])

str

In [13]:
df["trans_date_trans_time"] = pd.to_datetime(df["trans_date_trans_time"])

<ipython-input-13-7547bd998793>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["trans_date_trans_time"] = pd.to_datetime(df["trans_date_trans_time"])


In [14]:
df["time_var"] = [i.day_of_week for i in df["trans_date_trans_time"]]

<ipython-input-14-54b76ee89a3f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["time_var"] = [i.day_of_week for i in df["trans_date_trans_time"]]


In [15]:
df.loc[df['time_var'] < 5, 'time_var'] = 0
df.loc[df['time_var'] >= 5, 'time_var'] = 1
df.head()

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,trans_date_trans_time,gender,amt,city_pop,is_fraud,time_var
0,2020-06-21 12:14:25,M,2.86,333497,0,1
1,2020-06-21 12:14:33,F,29.84,302,0,1
2,2020-06-21 12:14:53,F,41.28,34496,0,1
3,2020-06-21 12:15:15,M,60.05,54767,0,1
4,2020-06-21 12:15:17,M,3.19,1126,0,1


In [16]:
df = pd.get_dummies(df, ["gender"])
df

,trans_date_trans_time,amt,city_pop,is_fraud,time_var,gender_F,gender_M
0,2020-06-21 12:14:25,2.86,333497,0,1,0,1
1,2020-06-21 12:14:33,29.84,302,0,1,1,0
2,2020-06-21 12:14:53,41.28,34496,0,1,1,0
3,2020-06-21 12:15:15,60.05,54767,0,1,0,1
4,2020-06-21 12:15:17,3.19,1126,0,1,0,1
...,...,...,...,...,...,...,...
555714,2020-12-31 23:59:07,43.77,519,0,0,0,1
555715,2020-12-31 23:59:09,111.84,28739,0,0,0,1
555716,2020-12-31 23:59:15,86.88,3684,0,0,1,0
555717,2020-12-31 23:59:24,7.99,129,0,0,0,1


In [17]:
df = df[['amt', 'city_pop', 'is_fraud','time_var', 'gender_F']]
df = df.rename(columns={'gender_F': 'gender', 'time_var': 'weekend'})
df

,amt,city_pop,is_fraud,weekend,gender
0,2.86,333497,0,1,0
1,29.84,302,0,1,1
2,41.28,34496,0,1,1
3,60.05,54767,0,1,0
4,3.19,1126,0,1,0
...,...,...,...,...,...
555714,43.77,519,0,0,0
555715,111.84,28739,0,0,0
555716,86.88,3684,0,0,1
555717,7.99,129,0,0,0


In [18]:
X = df[['amt', 'city_pop', 'weekend', 'gender']]
Y = df['is_fraud']
X.head()

,amt,city_pop,weekend,gender
0,2.86,333497,1,0
1,29.84,302,1,1
2,41.28,34496,1,1
3,60.05,54767,1,0
4,3.19,1126,1,0


# 4.) Train a Logistic regression.

In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [20]:
cutX = int((len(X) * .8) //1)
cutY = int((len(Y) * .8) //1)

In [21]:
X_train = X[:cutX]
X_test = X[cutX:]
Y_train = Y[:cutY]
Y_test = Y[cutY:]

In [22]:
scaler = preprocessing.StandardScaler().fit(X_train)
Xtrain_scale = scaler.transform(X_train)
Xtest_scale = scaler.transform(X_test)

In [23]:
log_reg = LogisticRegression().fit(X, Y)

In [24]:
log = LogisticRegression().fit(X_train, Y_train)

In [25]:
log.predict_proba(X_test)

array([[9.99182422e-01, 8.17578412e-04],
       [9.98838247e-01, 1.16175299e-03],
       [9.99181890e-01, 8.18109786e-04],
       ...,
       [9.99019689e-01, 9.80311274e-04],
       [9.87384880e-01, 1.26151200e-02],
       [9.88289784e-01, 1.17102156e-02]])

# 5.) The company you are working for wants to target at a False Positive rate of 5% what threshold should you use?

In [26]:
probs = log.predict_proba(X_test)

fpr, tpr, thresholds = roc_curve(Y_test, probs[:, 1])

threshold_index = np.abs(fpr - 0.05).argmin()
threshold = thresholds[threshold_index]
threshold

0.015278711585556206

In [27]:
Y_preds = (probs[:,1] > threshold).astype(int)

conf_matrix = confusion_matrix(Y_test, Y_preds)

print(conf_matrix)

[[105407   5553]
 [   120     64]]


# 6.) If the company makes .02*amt on True transactions and loses -amt on False (Use original data)

In [28]:
df_temp = df.copy()

In [29]:
df_temp["pred"] = log_reg.predict(X)

In [30]:
df_temp = df_temp[["pred", "is_fraud", "amt"]]

In [31]:
df_temp["factor"] = np.where(df_temp["pred"] == df_temp["is_fraud"], 0.02, -1)

In [32]:
df_temp["profit"] = df_temp["is_fraud"] * df_temp["factor"]

In [33]:
profit = sum(df_temp["profit"])
profit

-2145.0

# 7.) Using Logistic Regression Lasso to inform you. Would you use the selected features in a trusted prediction model?

In [34]:
lasso = LogisticRegression(penalty='l1', solver='saga').fit(X_train, Y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [35]:
probs_l1 = lasso.predict_proba(X_test)

fpr, tpr, thresholds = roc_curve(Y_test, probs_l1[:, 1])

threshold_index = np.abs(fpr - 0.05).argmin()
threshold = thresholds[threshold_index]
threshold

0.4823994359055558

In [36]:
Y_preds = (probs_l1[:,1] > threshold).astype(int)

conf_matrix = confusion_matrix(Y_test, Y_preds)

print(conf_matrix)

[[105538   5422]
 [   184      0]]


In [37]:
lasso.predict_proba(X_test)

array([[6.30434833e-01, 3.69565167e-01],
       [6.73573842e-01, 3.26426158e-01],
       [6.34407851e-01, 3.65592149e-01],
       ...,
       [8.40890477e-01, 1.59109523e-01],
       [5.14746492e-01, 4.85253508e-01],
       [1.00000000e+00, 2.50920219e-23]])

The model overpredict true positives and does not even predict true negatives. In addition the probabilities for 0 and 1 are 0, so I would not use the selected features in a trusted prediction model. 